# RNN text classification

In [14]:
import numpy as np
import os
import pandas as pd
import time

from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import tensorflow_datasets as tfds
print('tensorflow version: ', tf.__version__)

# 指定使用第一張GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

tensorflow version:  2.8.0


In [15]:
# # 上傳資料
# !wget -q https://github.com/TA-aiacademy/course_3.0/releases/download/v2.5_nlp/NLP_part3.zip
# !unzip -q NLP_part3.zip

In [16]:
output_dir = "Data"
zh_vocab_file = os.path.join(output_dir, "zh_vocab")
checkpoint_path = os.path.join(output_dir, "checkpoints.h5")

## Load Data

In [17]:
ptt_gossip = pd.read_csv('Data/ptt_gossip.csv')
ptt_gossip.drop(columns='idx', inplace=True)
print(ptt_gossip.shape)
ptt_gossip.head()

(7075, 2)


,sentence,label
0,反核人士最愛靠妖 核電廠蓋你家好不好，我當然說好 核廢料放我家好不好，我也ok 放在地下室就...,1
1,如標題， 今天去逛才看到的，如下圖所示: 位置在西屯區漢口路二段118號。 少了一個可以看...,1
2,新聞來源 2025非核家園，燃煤電廠30%、再生能源(綠能)20%、 天然氣發電50%的能...,1
3,牽了一台新的摩托車 車行老闆跟我說記得汽油要加95 還附帶 我開車行幾十年了 聽我的準沒錯 ...,1
4,各位30cm大大、F cup的水水，打給後 胎嘎後 本魯邊緣人，平日臉書沒朋友近日更4鬼怪肆...,1


## Filter sentence length

依照句子長度過濾

In [18]:
max_length = 256

ptt_gossip = ptt_gossip[ptt_gossip.sentence.str.len() < max_length]
ptt_gossip.reset_index(drop=True, inplace=True)
print(ptt_gossip.shape)
ptt_gossip.head()

(5091, 2)


,sentence,label
0,如標題， 今天去逛才看到的，如下圖所示: 位置在西屯區漢口路二段118號。 少了一個可以看...,1
1,牽了一台新的摩托車 車行老闆跟我說記得汽油要加95 還附帶 我開車行幾十年了 聽我的準沒錯 ...,1
2,各位30cm大大、F cup的水水，打給後 胎嘎後 本魯邊緣人，平日臉書沒朋友近日更4鬼怪肆...,1
3,是否有專板 ， 本板並非萬能問板 。 兩則 問卦， 自刪及被刪也算兩篇之內 ， 本看板嚴格禁...,1
4,亞洲盃男籃 中華隊被日本痛宰將近40分 87:49 想寫個慘字,1


## Train validation split

In [19]:
valid_size = 0.2
X_train, X_valid, y_train, y_valid = train_test_split(ptt_gossip['sentence'],
                                                      ptt_gossip['label'],
                                                      test_size=valid_size,
                                                      shuffle=True)

## Pre-processing

1. 將資料轉換成`tf.tensor`格式。
2. 使用`tfds.features.text.SubwordTextEncoder`進行斷詞，斷詞方式為`character-level`方式。

In [35]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))

In [36]:
%%time
try:
    tokenizer_zh = tfds.deprecated.text.SubwordTextEncoder.load_from_file(zh_vocab_file) 
    print('Load Chinese vocabulary: %s' % zh_vocab_file)
except: 
    print('Build Chinese vocabulary: %s' % zh_vocab_file)
    tokenizer_zh = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus((x.numpy() for x, y in train_dataset),
                                                                             max_subword_length=1,
                                                                             target_vocab_size=2**13)
    tokenizer_zh.save_to_file(zh_vocab_file)

Load Chinese vocabulary: Data/zh_vocab
CPU times: user 23.5 ms, sys: 15.7 ms, total: 39.3 ms
Wall time: 29.9 ms


In [37]:
print('Vocabulary size: ', tokenizer_zh.vocab_size)

Vocabulary size:  4217


In [38]:
tokenizer_zh

<SubwordTextEncoder vocab_size=4217>

### Example

In [39]:
sentence = '文瑋助教真壯'
token_id = tokenizer_zh.encode(sentence)

print('Sentence token_id: ', token_id)
print('Tokenization: ', [tokenizer_zh.decode([t]) for t in token_id])

Sentence token_id:  [63, 1756, 748, 352, 68, 1581]
Tokenization:  ['文', '瑋', '助', '教', '真', '壯']


## Convert to token_id

因為訓練時需要將每個字轉換成，這邊使用`.map`方式將`train_dataset`轉換成`token_id`。

In [40]:
def encode(sentence, label):
    zh_id = tokenizer_zh.encode(sentence.numpy())
    return (tf.cast(zh_id, tf.int32), tf.cast(label, tf.int32))

In [41]:
def tf_encode(sentence, label):
    """
    從encode輸出的zh_id不是Eager Tensor
    需要透過 tf.py_function 轉為Eager Tensor
    """
    return tf.py_function(encode, [sentence, label], [tf.int32, tf.int32])

In [42]:
train_dataset = train_dataset.map(tf_encode)
valid_dataset = valid_dataset.map(tf_encode)

In [43]:
tmp_valid = next(iter(valid_dataset))

In [44]:
pprint(tmp_valid)

(<tf.Tensor: shape=(65,), dtype=int32, numpy=
array([ 227,  254,  406,   79,  323,  369, 1241, 3993,    3,  127,    1,
        405,   12,  227,  254, 3993,   45,  431, 3993,  175,   58,  652,
        196, 3993,  281,  385,    8,  749,  134, 3993,  369, 1734, 3993,
        196,  483,  345,  258,    6,  272,   53,   10,  254,  151, 3993,
        106,   48,  112,  353,  128,    2,  104,  498,   33,    1,  104,
        498, 3993,  143,   44,  175,   65,   35,   29,  104,  498],
      dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>)


In [45]:
pprint(tokenizer_zh.decode(tmp_valid[0].numpy()))

'情歌王子張信哲 有名的四大情歌 過火 愛如潮水 別怕我傷心 信仰 水準海放一堆現在歌手 請問哪首才是經典中的經典 太想愛你也很經典'


## Input pipeline

這邊使用`tf.data.Data.from_tensor_slices`建立一個`generator`，每次訓練時讀取`batch_size`張圖片，通常會建立`generator`都是因為圖片量過大無法一次讀入記憶體，這邊使用`generator`是為了示範。

1. `.shuffle()`:進行`buffer_size`的打亂，每次從資料中取`buffer_size`個`batch`作為`buffer`，然後再從`buffer`中隨機抽一個`batch`出來做訓練，所以適當的`buffer_size`很重要，如果`buffer_size`過小會導致放在`buffer`裡的都是同一類別的圖片，最好的做法是直接把`buffer_size`設為訓練圖片數量(`len(X_train)`)，這樣能夠確保隨機性。

2. `.padded_batch()`:將每個`batch`進行`padding`，符合訓練的輸入格式。

3. `.repeat()`: 複製資料集為`epochs`份，訓練時需要`epochs`份

In [46]:
buffer_size = len(X_train)

embedding_size = 256
rnn_units = 512

batch_size = 64
epochs = 10

In [47]:
train_dataset = train_dataset.shuffle(buffer_size).padded_batch(batch_size, padded_shapes=([-1], []), drop_remainder=True).repeat(epochs)
valid_dataset = valid_dataset.padded_batch(batch_size, padded_shapes=([-1], []))

### Example

這邊使用`iter`呼叫`generator`來觀看其中一個`batch`。

In [48]:
tmp_generator = iter(train_dataset)
tmp_x, tmp_y = next(tmp_generator)

print('Sentence.shape: ', tmp_x.shape)
print(tmp_x)
print('-'*20)
print('Label.shape: ', tmp_y.shape)
print(tmp_y)

Sentence.shape:  (64, 243)
tf.Tensor(
[[ 66 880  79 ...   0   0   0]
 [ 66 129  40 ...   0   0   0]
 [ 53  10  51 ...   0   0   0]
 ...
 [ 12  30  17 ...   0   0   0]
 [768 170  12 ...   0   0   0]
 [ 71  22  14 ...   0   0   0]], shape=(64, 243), dtype=int32)
--------------------
Label.shape:  (64,)
tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1], shape=(64,), dtype=int32)


## Define LSTM model

`tensorflow2.0.0`預設是`eager model`，有助於在撰寫模型時`debug`以及觀看數值運算結果。

這裡使用`tf.keras`為基底進行建模，在`lstm`中需要注意輸入型態為`(timesteps, feature_size)`，另外常見有三個參數需要注意：

1. `embedding_size`: 每個字的詞向量大小。
2. `rnn_units`: `lstm`模型的神經元數量。
3. `return_sequences`: 是否輸出每個`timestep`的結果(`hidden_state`)，輸出型態為`(batch_size, )`。
4. `return_state`: 是否輸出最後一個`timestep`的結果(`hidden_state`和`cell_state`)。

其實`3.`和`4.`的功能有點重複了，通常我們只會拿最後一個`timestep`作為輸出，這邊我們將`return_sequences`設為`True`，並使用`slice`方式將最後一個`hidden_sate`拿出來。

最後使用`tf.keras.layers.Dense`輸出`2`個類別的概率。

In [49]:
def rnn_model(batch_size, rnn_units):
    input_layer = tf.keras.Input(shape=[None],batch_size=batch_size)
    embedding_layer = tf.keras.layers.Embedding(tokenizer_zh.vocab_size, embedding_size)(input_layer)
    
    lstm = tf.keras.layers.LSTM(units=rnn_units,
                                activation='tanh',
                                recurrent_activation='sigmoid',
                                use_bias=True,
                                return_sequences=True,
                                return_state=False,
                                recurrent_initializer='glorot_uniform')
    
    lstm_hidden_states = lstm(embedding_layer)
    
    lstm_last_state = lstm_hidden_states[:,-1,:]
    
    output = tf.keras.layers.Dense(2, activation='softmax', name='output')(lstm_last_state)
    
    return input_layer, output

In [50]:
input_layer, output = rnn_model(batch_size,rnn_units)
model = tf.keras.Model(inputs=input_layer, outputs=output)

In [51]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [52]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(64, None)]              0         
                                                                 
 embedding (Embedding)       (64, None, 256)           1079552   
                                                                 
 lstm (LSTM)                 (64, None, 512)           1574912   
                                                                 
 tf.__operators__.getitem (S  (64, 512)                0         
 licingOpLambda)                                                 
                                                                 
 output (Dense)              (64, 2)                   1026      
                                                                 
Total params: 2,655,490
Trainable params: 2,655,490
Non-trainable params: 0
___________________________________________________

In [53]:
history = model.fit(train_dataset,
                    epochs=epochs,
                    steps_per_epoch=len(X_train) // batch_size,
                    validation_data=valid_dataset,
                    validation_steps=len(X_valid) // batch_size)

Epoch 1/10


2023-09-20 17:53:41.828388: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


63/63 [==============================] - 8s 67ms/step - loss: 0.2688 - accuracy: 0.9459 - val_loss: 0.1662 - val_accuracy: 0.9625
Epoch 2/10
63/63 [==============================] - 6s 57ms/step - loss: 0.1729 - accuracy: 0.9601 - val_loss: 0.1613 - val_accuracy: 0.9625
Epoch 3/10
63/63 [==============================] - 6s 58ms/step - loss: 0.1689 - accuracy: 0.9603 - val_loss: 0.1620 - val_accuracy: 0.9625
Epoch 4/10
63/63 [==============================] - 6s 57ms/step - loss: 0.1701 - accuracy: 0.9601 - val_loss: 0.1604 - val_accuracy: 0.9625
Epoch 5/10
63/63 [==============================] - 6s 58ms/step - loss: 0.1693 - accuracy: 0.9598 - val_loss: 0.1610 - val_accuracy: 0.9625
Epoch 6/10
63/63 [==============================] - 6s 58ms/step - loss: 0.1698 - accuracy: 0.9598 - val_loss: 0.1620 - val_accuracy: 0.9625
Epoch 7/10
63/63 [==============================] - 6s 58ms/step - loss: 0.1699 - accuracy: 0.9601 - val_loss: 0.1649 - val_accuracy: 0.9625
Epoch 8/10
63/63 [======

In [54]:
model.save(checkpoint_path)

## Testing prediction

觀察`testing`的`precision, recall, f1-score`以及`confusion matrix`。

In [55]:
valid_pred = model.predict(valid_dataset)
valid_pred_id = np.argmax(valid_pred, axis=-1)
valid_true_id = np.array(y_valid)

In [56]:
print(classification_report(y_pred = valid_pred_id, y_true = valid_true_id))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        40
           1       0.96      1.00      0.98       979

    accuracy                           0.96      1019
   macro avg       0.48      0.50      0.49      1019
weighted avg       0.92      0.96      0.94      1019



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
confm = confusion_matrix(y_pred = valid_pred_id, y_true = valid_true_id)
pd.DataFrame(confm, index=['Actual_0', 'Actual_1'], columns=['Pred_0', 'Pred_1'])

,Pred_0,Pred_1
Actual_0,0,40
Actual_1,0,979
